In [60]:
import folium
import pandas as pd
import geopandas as gpd
from folium import plugins

# Initialize folium map centered in haverhill's coordinates

Initialize a map of Haverhill on the Folium Platform

In [61]:
haverhill = folium.Map(
    location = [42.7762,-71.0773],
    zoom_start = 13)

Create a Border of all wards/precincts. 

In [62]:
feature_wp = folium.FeatureGroup(name='Wards/Precincts')

df = gpd.read_file('Wards/Hav_Precincts_Wards_WGS84.shx')
df = df.to_crs(epsg ='4326')
js = df.to_json()

style_function = lambda x: {'color': 'black',
                           'weight': 2.0,
                           'fillOpacity':0}
feature_wp = folium.GeoJson(js,name = 'wards/precincts',
               style_function = style_function)

Border of CDBG

In [63]:
feature_cdbg = folium.FeatureGroup(name='CDBG_area')

df = gpd.read_file('CDBG_area/Hav_CDBG_Area_WGS84.shx')
df = df.to_crs(epsg ='4326')
js = df.to_json()

style_function = lambda x: {'color': 'red',
                           'weight': 1.5,
                           'fillOpacity':0.2}
feature_cdbg = folium.GeoJson(js,name = 'CDBG',
               style_function = style_function)

Trace Refuse_Route

In [64]:
feature_rr = folium.FeatureGroup(name='Refuse_routes')

df = gpd.read_file('Refuse_route/Hav_Refuse_Routes_WGS84.shx')
df = df.to_crs(epsg ='4326')
js = df.to_json()

style_function = lambda x: {'color': 'green',
                           'weight': 2.5,
                           'fillOpacity':0}
feature_rr = folium.GeoJson(js,name = 'Refuse_routes',
               style_function = style_function)

In [65]:
feature_wp.add_to(haverhill)
feature_cdbg.add_to(haverhill)
feature_rr.add_to(haverhill)

# Paint wards based on number of requests

In [66]:
from branca.colormap import linear
data = pd.read_csv("department.csv")
ward_df = gpd.read_file('Wards/Hav_Precincts_Wards_WGS84.json',driver = 'GeoJSON')
ward_df = ward_df.to_crs(epsg ='4326')
ward_df = ward_df[['Precinct','geometry']]
data_df = data[['Request_ID','Ward_Precinct','Longitude','Latitude']]

Merge request data with their respective precincts

In [67]:
feature_colormap = folium.FeatureGroup(name='Precincts')

count_df = data_df.groupby('Ward_Precinct')['Request_ID'].nunique().reset_index()
count_df.rename(columns={'Ward_Precinct':'Precinct'}, inplace=True)
count_df.rename(columns={'Request_ID':'nb'}, inplace=True)

"""Using the 2011 map data"""
ward_df = ward_df.tail(21)
nbh_geo_count_df = ward_df.sort_values(by = 'Precinct').reset_index()
nbh_geo_count_df['nb'] = count_df['nb']
nbh_geo_count_df = nbh_geo_count_df[['Precinct','geometry','nb']]
nbh_geo_count_df.head()

,Precinct,geometry,nb
0,1-1,"POLYGON ((-71.09160 42.77516, -71.09232 42.774...",734
1,1-2,"POLYGON ((-71.08679 42.78449, -71.08710 42.785...",706
2,1-3,"POLYGON ((-71.10464 42.79081, -71.10478 42.790...",904
3,2-1,"POLYGON ((-71.06884 42.77243, -71.06980 42.773...",815
4,2-2,"POLYGON ((-71.12075 42.76855, -71.12138 42.768...",860


In [68]:
nbh_count_colormap = linear.YlGnBu_09.scale(min(count_df['nb']),
                                            max(count_df['nb']))

style_function = lambda x: {
    'fillColor': nbh_count_colormap(x['properties']['nb']),
    'color': 'black',
    'weight': 1.5,
    'fillOpacity': 0.7
}

feature_colormap = folium.GeoJson(
    nbh_geo_count_df,name = 'number of requests',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['Precinct', 'nb'],
        aliases=['Precinct id', 'Number of Requests'],
        localize=True
    )
)

feature_colormap.add_to(haverhill)
folium.LayerControl(collapsed=False).add_to(haverhill)
haverhill.save('overview.html')

# Requests per department visualization

In [55]:
data.head()

,Request_ID,Create_Date,Request_Type_ID,Request_Type,Department,Street_ID,Longitude,Latitude,Ward_Precinct,geometry
0,64070,4/4/19 12:56,310,LAZ - Downtown Parking Issue,311 Call Center,1132,-71.079045,42.774499,3_1,POINT (-71.07905 42.77450)
1,77191,9/20/19 21:46,71825,Utilities Other,311 Call Center,872,-71.148356,42.781949,5_2,POINT (-71.14836 42.78195)
2,21359,10/30/17 13:34,311,Parking Meter Malfunction,311 Call Center,1035,-71.082849,42.773652,1_2,POINT (-71.08285 42.77365)
3,27735,2/5/18 14:21,1980,Train Idling Compaint,311 Call Center,851,-71.075890,42.769535,2_1,POINT (-71.07589 42.76953)
4,21212,10/26/17 18:47,310,LAZ - Downtown Parking Issue,311 Call Center,1036,-71.094576,42.770090,2_3,POINT (-71.09458 42.77009)


In [56]:
from shapely.geometry import Point,shape
locs_geometry = [Point(xy) for xy in zip(data.Longitude,data.Latitude)]
crs = {'init':'epsg:4326'}
locs_gdf = gpd.GeoDataFrame(data,crs = crs,geometry= locs_geometry)

In [57]:
loc_map = folium.Map(
    location = [42.7762,-71.0773],
    zoom_start = 13)

style_function = lambda x: {
    'fillColor': 'grey',
    'color': 'black',
    'weight': 1,
    'fillOpacity': 0.2
}

folium.GeoJson(
    nbh_geo_count_df,
    name = 'Precinct Base Map',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['Precinct', 'nb'],
        aliases=['Precinct', 'Number of Requests'],
        localize=True
    )
).add_to(loc_map)

folium.map.CustomPane('labels').add_to(loc_map)
loc_map

In [58]:
feature_311 = folium.FeatureGroup(name = '311')
feature_aa = folium.FeatureGroup(name = 'animal_control')
feature_cc = folium.FeatureGroup(name = 'city_clerk')
feature_f = folium.FeatureGroup(name = 'fire')
feature_fp = folium.FeatureGroup(name = 'fire_police')
feature_h = folium.FeatureGroup(name = 'highway')
feature_is = folium.FeatureGroup(name = 'inspectional_services')
feature_m = folium.FeatureGroup(name = 'mayor')
feature_p = folium.FeatureGroup(name = 'planning')
feature_po = folium.FeatureGroup(name = 'police')
feature_ss = folium.FeatureGroup(name = 'stormwater_sewer')
feature_t = folium.FeatureGroup(name = 'treasurer')
feature_water = folium.FeatureGroup(name = 'water')
feature_wb = folium.FeatureGroup(name = 'water_billing')
feature_ww = folium.FeatureGroup(name = 'water_wastewater_department')

for i,v in locs_gdf.iterrows():
    
    if v['Department'] == '311 Call Center':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_311)
    if v['Department'] == 'Animal control':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_aa)
    if v['Department'] == 'City Clerk':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_cc)
    if v['Department'] == 'Fire':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_f)
    if v['Department'] == 'Fire & Police':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_fp)
    if v['Department'] == 'Highway':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_h)
    if v['Department'] == 'Inspectional Services':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_is)
    if v['Department'] == 'Mayor':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_m)
    if v['Department'] == 'Planning':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_p)
    if v['Department'] == 'Police':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_po)
    if v['Department'] == 'Stormwater/Sewer':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_ss)
    if v['Department'] == 'Treasurer':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_t)
    if v['Department'] == 'Water':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_water)
    if v['Department'] == 'Water Billing':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_wb)
    if v['Department'] == 'Water/Wastewater Department':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_ww)

feature_311.add_to(loc_map)
feature_aa.add_to(loc_map)
feature_cc.add_to(loc_map)
feature_f.add_to(loc_map)
feature_fp.add_to(loc_map)
feature_h.add_to(loc_map)
feature_is.add_to(loc_map)
feature_m.add_to(loc_map)
feature_p.add_to(loc_map)
feature_po.add_to(loc_map)
feature_ss.add_to(loc_map)
feature_t.add_to(loc_map)
feature_water.add_to(loc_map)
feature_wb.add_to(loc_map)
feature_ww.add_to(loc_map)
folium.LayerControl(collapsed=False).add_to(loc_map)

add a layer of labels

In [59]:
loc_map.save('requests_per_department.html')